<a href="https://colab.research.google.com/github/AliaksandrSiarohin/first-order-model/blob/master/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for paper "Motion Representations for Articulated Animation"

**Clone repository**

In [1]:
git clone https://github.com/AliaksandrSiarohin/motion-articulated

SyntaxError: invalid syntax (<ipython-input-1-23754544b7a8>, line 1)

In [2]:
cd motion-articulated

[Errno 2] No such file or directory: 'motion-articulated'
/home/gin/fomm-private2


**Mount your Google drive folder on Colab**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.**

**Load driving video and source image**

In [4]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('sup-mat/source.png')
driving_video = imageio.mimread('sup-mat/driving.mp4')


source_image = resize(source_image, (384, 384))[..., :3]
driving_video = [resize(frame, (384, 384))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [5]:
from demo import load_checkpoints

generator, region_predictor, avd_network = load_checkpoints(config_path='config/ted384.yaml',
                                                            checkpoint_path='ted384.pth')

**Perfrorm image animation**

In [6]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, 
                             region_predictor, avd_network, animation_mode='avd')

#save resulting video
imageio.mimsave('generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 133/133 [00:50<00:00,  2.65it/s]
